In [1]:
import datetime
import random as rn
import psycopg as c
from validate_email_address import validate_email
from verify_email import verify_email
import os
import matplotlib.pyplot as plt
import smtplib # for mail only 
from email.message import EmailMessage  # to senf email to already exiting mail
import sys  # to exit code 
import numpy as np

class Account:
    con=c.connect(host="localhost",dbname="Bank_management",user="postgres",password="jeel",port=5433)
    cur=con.cursor()
    def __init__(self):
                self.main()            
    def main(self):
            print('''Select From Below:
                    1.Open an Account
                    2.Transaction
                    3.Display Account Details
                    4.Delete an Account 
                    5.View Balance
                    6.Change details
                    7.Graph
                    8.Exit
            ''')
            try:
                choice1=int(input("Enter Your choice  : " ))
                try:
                    if choice1==8:
                        print("Thank You for using our Bank")
                        sys.exit()
                    else:
                        if choice1==1:
                            self.open_account()
                        elif choice1==2:
                            self.trans()
                        elif choice1==3:
                            self.display()
                        elif choice1==4:
                            self.delete()
                        elif choice1==5:
                            self.view_bal()
                        elif choice1==6:
                            self.change_details()
                        elif choice1==7:
                            self.graph_show()
                except SystemExit:
                        print("Exiting System")
            except ValueError as e:
                print("Enter a number ",e)
                    
# """----------------------------------------------------------------------------------------------------------------------"""
    def graph_show(self):
        enter_acc = int(input("Enter your account number: "))
        sql_fetchTrans = "SELECT curr_bal FROM trans_hist WHERE acct_no=%s"
        self.cur.execute(sql_fetchTrans, (enter_acc,))
        li = []

        fetch_det = self.cur.fetchall()  # Fetch all rows for the provided account number

        if fetch_det:  # Check if there are any rows fetched
            for row in fetch_det:  # Iterate over each fetched row
                li.append(row[0]) # Assuming curr_bal is the first column in the result set
        else:
            print("No transaction history found for the provided account number.")
        print("The required transaction graph :")
        li_new=np.array(li)
        plt.plot(li_new)
        plt.show()
        self.main()

    
    def open_account(self):
        try:

            self.name=input("Enter Your name : ")
            self.acc_no=rn.randint(99999,1000000)
            f=open(f"{self.acc_no}.txt","w")
            f.write(f"Name  : {self.name} \n")
            f.write(f"Account no : {self.acc_no}\n")

            while True:
                try:
                    self.mobile_no=int(input("Enter Your Mobile Number : "))
                except ValueError as e:
                    print("An error occured",e)
                if len(str(self.mobile_no))!=10:
                    print("Please Enter 10 digit Valid mobile Number")
                else:
                    break;
            f.write(f"Mobile Number : {self.mobile_no} \n")

            print("Provide us with your Date of Birth Details : ")
    #         cur_date=date.today()

            while True:
                try:
                    year=int(input("Enter Year :"))
                    month=int(input("Enter Month : "))
                    day=int(input("Enter Date : "))
                    self.dob=datetime.date(year,month,day)
                    break;
                except Exception as e:
                         print(e)

            f.write(f"Birth Date : {self.dob} \n")
            self.Adrs=input("Enter Your Adress : ")
            f.write(f"Address : {self.Adrs} \n")
            while True:
                try:
                    self.open_bal=int(input("Enter The opening amount of the account  :"))
                except ValueError as e:
                    print("Enter a number ",e)
                    continue
                if self.open_bal<0:
                    print("Enter Valid Balance ")
                else:
                    break

            while True:
                    try:
                        self.adhar_no=int(input("Enter your adhar number :"))
                    except ValueError as e:
                        print("Enter a number ",e)
                        continue
                    if len(str(self.adhar_no))!=12:
                        print("Enter Valid Adhar card Number")
                    else:
                        break
            f.write(f"Adhar crad Number : {self.adhar_no} \n")
            while True:
                self.email=input("Enter Your Email")
                is_valid = validate_email(self.email)
                if is_valid:
                    break;
                else:
                    print("Enter Valid Email")
            f.write(f"Email : {self.email} \n")
            f.write("                          TRANSACTION DETAILS       \n")
            self.curr_bal=self.open_bal
            print(f"Your Account No is : {self.acc_no} \n")
            sql1="insert into acnt_detail values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            sql_login='insert into login_detail values(%s,%s)'
            var=(self.acc_no,self.name,self.dob,self.mobile_no,self.open_bal,self.curr_bal,self.adhar_no,self.Adrs,self.email)
            self.cur.execute(sql1,var)
            self.cur.execute(sql_login,(self.acc_no,self.mobile_no,))
            self.con.commit()

            print(f"""User Login Details :
                    username:{self.acc_no}
                    password:{self.mobile_no}
            """)
            
            print("Your Account Has Opened Succesfully")
            if __name__=='__main__':
                self.Email_Send("Opening of your new Account",f"""
                    THANK YOU
                    for opening a bank account in our bank,
                    Name: {self.name}
                    Account Number: {self.acc_no}
                    User Login Details :
                    username:{self.acc_no}
                    password:{self.mobile_no}
                """,self.email)
            f.close()
            self.main()
        except Exception as e:
            print("An error occured while opening a account")


# """----------------------------------------------------------------------------------------------------------------------""
    def deposite_amt(self,account_no,amt):
            current_date = datetime.datetime.now()
            formatted_date = current_date.strftime('%Y-%m-%d')
            cur_direct=os.getcwd()
            file_path = os.path.join(cur_direct,f"{account_no}.txt")
            if os.path.isfile(file_path):
                f=open(f"{account_no}.txt","a+")
                sql='select cur_bal,email from acnt_detail where acc_no=%s'
                self.cur.execute(sql,(account_no,))
                fetch=self.cur.fetchone()
                if fetch:
                    x=fetch[0]
                    email_fetched=fetch[1]
#                     print(email_fetched)
                    new_bal=x+amt
                    f.write(f"transaction {datetime.datetime.now()}    previous balance  : {x} ,  New Balance : {new_bal} , The Amount Deposited  : {amt} \n")
                    f.close()    
                    sql1="update acnt_detail set cur_bal=%s where acc_no=%s"
                    sql2="insert into trans_hist values(%s,%s,%s,%s)";
                    new_balance=(new_bal,account_no)
                    
                    self.cur.execute(sql1,new_balance)
                    print(f"transaction {datetime.datetime.now()} old amount {x} , new Amoount {new_bal} ")
                    if __name__=='__main__':
                        self.Email_Send(f"Transaction on account number :{account_no}",f"""
                            Transaction Details:
                            Time: {datetime.datetime.now()} 
                            Old amount: {x}
                            New Amoount: {new_bal}
                            Amount Deposited: {amt}
                        """,email_fetched)

                    var=(account_no,formatted_date,x,new_bal)
                    self.cur.execute(sql2,var)
                    self.con.commit()
                self.trans()
            else:
                print("Account Not found")
                
    def withdraw_amt(self,ac_dep_no,dep_amt):
            current_date = datetime.datetime.now()
            formatted_date = current_date.strftime('%Y-%m-%d')
            cur_direct=os.getcwd()
            file_path = os.path.join(cur_direct,f"{ac_dep_no}.txt")
            if os.path.isfile(file_path):
                f=open(f"{ac_dep_no}.txt","a+")
                sql='select cur_bal,email from acnt_detail where acc_no=%s'
                self.cur.execute(sql,(ac_dep_no,))
                fetch=self.cur.fetchone()
                if fetch:
                    x=fetch[0]
                    email_fetch=fetch[1]
                    if dep_amt>x:
                        print("Insufficient Balance")
                    else:
                        new_bal=x-dep_amt
                        sql1="update acnt_detail set cur_bal=%s where acc_no=%s"
                        sql2="insert into trans_hist values(%s,%s,%s,%s)";
                        new_balance=(new_bal,ac_dep_no)
                        self.cur.execute(sql1,new_balance)
                        var2=(ac_dep_no,formatted_date,x,new_bal)
                        self.cur.execute(sql2,var2)
                        print(f"Amount withdrawn Succesfully , old amount {x} , new Amoount {new_bal} ")
                        if __name__=='__main__':
                            self.Email_Send(f"Transaction on account number :{ac_dep_no}",f"""
                                Transaction Details:
                                Time: {datetime.datetime.now()} 
                                Old amount: {x}
                                New Amoount:  {new_bal}
                                Amount withdrawn: {dep_amt}
                            """,email_fetch)

                        
                        f.write(f"transaction {datetime.datetime.now()}  old amount {x} , new Amoount {new_bal} , Amount withdrawn : {dep_amt} \n")
                        self.con.commit()
                        f.close()
                    self.trans()
            else:
                print("Account Not found")




    def trans(self):
        current_date = datetime.datetime.now()
        formatted_date = current_date.strftime('%Y-%m-%d')
        print('''Select from below :
                1.Deposite Amount
                2.Withdraw Amount
                3.Back to Main menu
        ''')
        try:
            am_choice=int(input("Enter your choice '1' or '2' "))
        except ValueError as e:
            print("Enter a number ",e)
            self.trans()
        if am_choice==1:
            try:
                dep_amt=int(input("Enter Amount to be deposited : "))
                ac_dep_no=int(input("Enter Your account number  : "))
                self.deposite_amt(ac_dep_no,dep_amt)
            except ValueError as e:
                print("Enter a number ",e)
                self.trans()
            
        elif am_choice==2:
            try:
                dep_amt=int(input("Enter Amount to be WithDrawn : "))
                ac_dep_no=int(input("Enter Your account number  : "))
                self.withdraw_amt(ac_dep_no,dep_amt)
            except ValueError as e:
                print("Enter a number ",e)
                self.trans()
        elif am_choice==3:
            self.main()
        
        else:
            print("Enter Valid choice")
            self.trans()

# """----------------------------------------------------------------------------------------------------------------------""
    def display(self):
        acc_dis=int(input("Enter the account number whose details wanted to be displayed :"))
        sql="select * from acnt_detail where acc_no=%s"
        self.cur.execute(sql,(acc_dis,))
        result=self.cur.fetchone()
        if result:
            print(f"Account Number : {result[0]}")
            print(f"Name           : {result[1]}")
            print(f"Date of Birth  : {result[2]}")
            print(f"Mobile number  : {result[3]}")
            print(f"Curent Balnace : {result[5]}")
            print(f"Adhar Number   : {result[6]}")
            print(f"Adress         : {result[7]}")
            self.main()
        else:
            print("Account Not found")
            self.main()
        self.con.commit()
        

# """----------------------------------------------------------------------------------------------------------------------""
    def delete(self):
        try:
            acc_dele = int(input("Enter the account number which is to be deleted: "))
        except ValueError as e:
                print("Enter a number ",e)
                self.delete()
        sqlF= "select acc_no from acnt_detail where acc_no=%s"
        self.cur.execute(sqlF, (acc_dele,))
        x = self.cur.fetchone()
        if x:
            print("Account found. Attempting deletion...")
            sql = "delete from acnt_detail where acc_no=%s"
            sql2 = "delete from login_detail where u_id=%s"
            otp_delete=rn.randint(9999,100000)
            sql_email="select email from acnt_detail where acc_no=%s"
            self.cur.execute(sql_email,(acc_dele,))
            email_f=self.cur.fetchone()
            print("Email fetched-----------------------------",email_f[0])
            if __name__=='__main__':
                        self.Email_Send("Deleting of your account",f"""
                            OTP for account delteing :{otp_delete}
                        """,email_f[0]) 
            try:
                otp_f=int(input("Enter the OTP for deleting your account : "))
                if otp_f==otp_delete:
                    result = self.cur.execute(sql, (acc_dele,))
                    r2= self.cur.execute(sql2, (acc_dele,))
                    self.con.commit()
                    if result.rowcount > 0 and r2.rowcount>0:
                        print(f"Account with account number {acc_dele} is deleted.")
                        if __name__=='__main__':
                            self.Email_Send("Deleting of your account",f"""
                                THANK YOU for staying with us,Account with:
                                Account no :{acc_dele}
                                has been deleted
                            """,email_f[0])
                        self.main()

                    else:
                        print(f"Failed to delete account with account number: {acc_dele}. No rows affected.")
                else:
                    print("Entered OTP is wrong")
            except Exception as e:
                print("Error occurred during deletion:", e)
                self.main()
        else:
            print("Account Not found")
            self.main()

# """----------------------------------------------------------------------------------------------------------------------""

            
    def view_bal(self):
        try:
            acc_dis=int(input("Enter the account number whose Balance you wanted to be displayed :"))
        except ValueError as e:
                print("Enter a number ",e)
                self.view_bal()
        sql="select * from acnt_detail where acc_no=%s"
        self.cur.execute(sql,(acc_dis,))
        result=self.cur.fetchone()
        if result:
            print(f"Current Balance  : {result[5]}")
            print(f"Openning Balance : {result[4]}")
            self.main()
        else:
            print("Account not found")
            self.view_bal()
            
        self.con.commit()
        
        
# ---------------------------------------------------------------------------------------------------------------------------
    def change_details(self):
        try:
            ac_no = int(input("Enter the account number whose details need to be changed: "))
        except Exception as e:
            print("An error occurred: ", e)
            self.change_details()
            return  # Exit the method

        cur_direct = os.getcwd()
        file_path = os.path.join(cur_direct, f"{ac_no}.txt")

        if os.path.isfile(file_path):
            try:
                f=open(f"{ac_no}.txt", "a+")
                f.write("_____________________________UPDATED DETAILS______________________________\n")
                while True:
                        choice = input("""
                            1. Change Name
                            2. Change Phone Number
                            3. Change Address
                            4. Change Email
                            5. Change Password
                            6. Exit
                        """)

                        if choice == "1":
                            name = input("Enter New name: ")
                            sql_name = "UPDATE acnt_detail SET name_t=%s WHERE acc_no=%s"
                            try:
                                self.cur.execute(sql_name, (name, ac_no))
                                print("Name Updated Successfully")
                                f.write(f"Name : {name}\n")
                                self.con.commit()
                                f.close()
                            except Exception as e:
                                print(f"Error updating name: {e}")

                        elif choice == "2":
                            try:
                                mobile_no=int(input("Enter Your Mobile Number : "))
                            except ValueError as e:
                                print("An error occured",e)
                            if len(str(mobile_no))!=10:
                                print("Please Enter 10 digit Valid mobile Number")
                            else:
                                sql_mob="update acnt_detail set mob_no=%s where acc_no=%s"
                                try:
                                    self.cur.execute(sql_mob,(mobile_no,ac_no,))
                                    f.write(f"Mobile Number Updated  : {mobile_no} \n")
                                    print("MObile Number Updated Succesfully")
                                    self.con.commit()
                                    f.close()
                                except Exception as e:
                                    print(f"Error updating name: {e}")

                        elif choice == "3":
                            adrs=input("Enter Your new Adres")
                            sql_adrs="update acnt_detail set adrs=%s where acc_no=%s"
                            try:
                                self.cur.execute(sql_adrs,(adrs,ac_no,))
                                f.write(f"Updated Address  : {adrs} \n")
                                print("Updated Address Succesfully: ",adrs)
                                self.con.commit()
                                f.close()
                            except Exception as e:
                                print(f"Error updating name: {e}")

            

                        elif choice == "4":
                            email=input("Enter Your Email")
                            is_valid = validate_email(email)
                            if is_valid:
                                sql_email="update acnt_detail set email=%s where acc_no=%s"
                                try:
                                    self.cur.execute(sql_email,(email,ac_no,))
                                    f.write(f"Email : {email} \n")
                                    print("Email Updated Succesfully")
                                    self.con.commit()
                                    f.close()
                                except Exception as e:
                                    print(f"Error updating name: {e}")
                            else:
                                print("Enter Valid Email")
                                self.change_details()
                        elif choice=="5":
                            try:
                                old_pwd=int(input("Enter old password : "))
                                sql_pwd='select pwd from login_detail where u_id=%s'
                                self.cur.execute(sql_pwd,(ac_no,))
                                r=self.cur.fetchone()
                                if old_pwd==r[0]:
                                    otp_pwd=rn.randint(9999,100000)
                                    sql_email="select email from acnt_detail where acc_no=%s"
                                    self.cur.execute(sql_email,(ac_no,))
                                    email_f=self.cur.fetchone()
#                                     print("Email fetched-----------------------------",email_f[0])
                                    if __name__=='__main__':
                                                self.Email_Send("Updating Password OTP",f"""
                                                    OTP for Updating password :{otp_pwd}
                                                """,email_f[0]) 
                                        
                                    otp_verify=int(input("Enter OTP :"))
                                    if otp_verify==otp_pwd:
                                        new_pwd=int(input("Enter New Password"))
                                        sql_set_pwd="update login_detail set pwd=%s where u_id=%s"
                                        self.cur.execute(sql_set_pwd,(new_pwd,ac_no,))
                                        f.write(f"Password Updated : {new_pwd} \n")
                                        self.con.commit()
                                        f.close()
                                    else:
                                        print("Wrong OTP entered")
                                else:
                                    print("No such password found")     
                            except Exception as e:
                                print("Error Occured : ",e)
                                self.change_details()
                        elif choice == "6":
                            print("Exiting change_details")
                            self.main()  # No need to break here
                            # break  # No need for this break statement

                        else:
                            print("Please Enter a Valid Input")

            except Exception as e:
                print("An error occurred while processing: ", e)
                return

            finally:
                self.con.commit()
                f.close()
        else:
            print("No such account exists")
            self.change_details()

# """----------------------------------------------------------------------------------------------------------------------""
    def Email_Send(self,subject,body,to):
        msg=EmailMessage()
        msg.set_content(body)
        msg['subject']=subject
        msg['to']=to
        user='jeelbpala0125@gmail.com'
        password='jcvn laqx aowr uewk'
        msg['from']=user
        server=smtplib.SMTP("smtp.gmail.com",587)
        server.starttls()
        server.login(user,password)
        server.send_message(msg)
        server.quit()

        
        
class Account_login(Account):
    def __init__(self):
#         self.con = super.con
        self.cur = self.con.cursor()
#         self.logged_in_user = None
        print("Welcome to our Bank")
        log=(input("Enter 1 to login or 2 to sign up or anything to exit"))
        if log=='1':
            self.login()
        elif log=='2':
            self.sing_in()
        else:
            try:
                sys.exit()
            except SystemExit:
                print("Exiting System")
    def login(self):
        try:
            username=input("Enter your user name :")
            password=int(input("Enter your password [must be digits] :"))
        except Exception as e:
                print("Enter a number ",e)
                self.login()
        sql_login='select * from login_detail where u_id=%s'
        self.cur.execute(sql_login,(username,))
        fetch=self.cur.fetchone()
        if fetch:
            print("User found")
            pwd_fetched=fetch[1]
            if pwd_fetched==password:
                print("Logged in succesfully")
                self.main()
            elif pwd_fetched!=password:
                count=1
                while count<3:
                    password=input("Enter your password [must be digits] :")
                    if pwd_fetched==password:
                        self.main()
                        break
                    else:
                        print("Wrong password enterd")
                        count+=1
                    if count==3:
                        print("Sorry your trials are over come back after some time")
                        try:
                            sys.exit()
                        except SystemExit:
                            print("Exiting System")
            else:
                print("Wrong Password")
                pwdInp=input("For Reseting password press 1 or anything else to exit :")
                if pwdInp=='1':
                    emailReset=input("Enter Your email id :")
                    otp=rn.randint(99999,1000000)
                    if __name__=='__main__':
                        self.Email_Send("Reseting of your password",f"""OTP [one time password] for Resting your password : {otp}"""
                                        ,emailReset)
                        count=0
                        while count<3:
                            try:
                                userInp=int(input("Enter the OTP send to your mail :"))
                            except ValueError as e:
                                print("Enter a number ",e)
                                continue
                            if userInp==otp:
                                new_pwd=int(input("Enter a new password [only digits]"))
                                sql_update_pwd='update login_detail set  pwd=%s where u_id=%s'
                                var2=(new_pwd,username)
                                self.cur.execute(sql_update_pwd,var2)
                                self.con.commit()
                                break
                            else:
                                print("Wrong OTP entered")
                                count+=1
                                print(f"{3-count} number of attemps left")
                            if count==3:
                                print("Sorry Your 3 Attemps are over please try again after 24 hrs ")
                                try:
                                    sys.exit()
                                except:
                                    print("Exiting system")
                else:
                    try:
                        sys.exit()
                    except:
                        print("Exiting system")
                    
                self.login()
            
        else:
            print("No such user found")
            self.sing_in()
    def sing_in(self):
        while True:
            print("Welcome to our Bank")
            print("For signing in User must have account in our bank!")
            uInp=(input("""
                Do you have a account or want to create one ? 
                Press 1 to create account or press anything if already have a account
                Or if you don't want to create account press 0 
            """ ))
            if uInp=='1':
                self.open_account()
                self.login()

            elif uInp=='0':
                print("Sorry to let you go but we can't procced further Thank you for using our Bank")
                break
            else:
                print("Please Log in first to get started with our bank")
                self.login()

       
        
a= Account_login()



        


OperationalError: connection failed: could not receive data from server: Socket is not connected (0x00002749/10057)
could not send SSL negotiation packet: Socket is not connected (0x00002749/10057)